In [20]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [21]:
from my_functions import drug_name_match
from my_functions import drug_name_filter_match

In [22]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [23]:
db_df = pd.read_pickle('../Pickles/db_df.pkl')
grx_df = pd.read_pickle('../Pickles/grx_df.pkl')
nadac_df = pd.read_pickle('../Pickles/nadac_df.pkl')

In [24]:
# created new column chem_name for grx_df to join with chem_name in db_df.
# looping matches between grx_df['drug_name'] and db_df['brand_names'].
for i, grx_drug in enumerate(grx_df['drug_name']):
    for z, db_brand in enumerate(db_df['brand_names']):
        if grx_drug in db_brand:
            grx_df.loc[i, 'chem_name'] = db_df['chem_name'][z]

In [25]:
# made all null values for grx_df['chem_name'] 'No Match'
# to clearly identify rows that have not yet been matched with a chemical.
grx_df.loc[grx_df['chem_name'].astype(str) == 'nan', 'chem_name'] = 'No Match'
        
# checking for the number of rows that have not yet been matched with a chemical.
len(grx_df[grx_df['chem_name'] == 'No Match'])

1816

In [26]:
# looping matches between grx_df['generic_name'] and db_df['brand_names'].
drug_name_filter_match(grx_df, 'chem_name', 'generic_name', db_df, 'brand_names')

# looping matches between grx_df['drug_name'] and db_df['generic_names'].
drug_name_filter_match(grx_df, 'chem_name', 'drug_name', db_df, 'generic_names')

# looping matches between grx_df['generic_name'] and db_df['generic_names'].
drug_name_filter_match(grx_df, 'chem_name', 'generic_name', db_df, 'generic_names')

# looping matches between grx_df['drug_name'] and db_df['chem_name'].
drug_name_filter_match(grx_df, 'chem_name', 'drug_name', db_df, 'chem_name')

# looping matches between grx_df['chem_name'] and db_df['chem_names'].
drug_name_filter_match(grx_df, 'chem_name', 'generic_name', db_df, 'chem_name')

# checking for the number of rows that have not yet been matched with a chemical.
print(len(grx_df[grx_df['chem_name'] == 'No Match']))

1102


In [27]:
# created new dataframe to join grx_df drugs with their chemical equivalents in db_df.
df = pd.merge(grx_df, db_df, on = 'chem_name')

In [28]:
# created new column ndc_drug_name for grx_df to join with the equivalent drug in nadac_df.
# looping matches between df['drug_name'] and nadac_df['ndc_drug_name'].
drug_name_match(df, 'ndc_drug_name', 'drug_name', nadac_df, 'ndc_drug_name')

# made all null values for df['ndc_drug_name'] 'No Match'
# to clearly identify rows that have not yet been matched with a chemical.
df.loc[df['ndc_drug_name'].isnull(), 'ndc_drug_name'] = 'No Match'

# checking for the number of rows that have not yet been matched with a nadac equivalent drug.
print(len(df[df['ndc_drug_name'] == 'No Match']))

1042


In [29]:
# filled in more values for ndc_drug_name to join with df with the equivalent drug in nadac_df.
# looping matches between df['generic_name'] and nadac_df['ndc_drug_name'].
drug_name_filter_match(df, 'ndc_drug_name', 'generic_name', nadac_df, 'ndc_drug_name')

# checking for the number of rows that have not yet been matched with a nadac equivalent drug.
print(len(df[df['ndc_drug_name'] == 'No Match']))

576


In [31]:
# replace current dataframe with version joining drugs between df and nadac_df.
df = pd.merge(df, nadac_df, on = 'ndc_drug_name')

# dropped columns from the dataframe that will no longer be needed for analysis or identification.
df.drop(['page_url', 'ndc_drug_name', 'brand_names', 'generic_names'], axis = 1, inplace = True)

# pickle final version of merged_df thatincludes nadac data.
df.to_pickle('../Pickles/merged_df.pkl')

In [32]:
# checked for number of drugs remaining for analysis.
len(df)

1780

In [ ]:
df.info()